In [1]:
import requests, time, math, threading
import sqlite3, os
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
from queue import Queue
from tools import get_chrome, find_element, get_soup
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By

In [2]:
def get_chrome(url, chromeDriver= r'C:\webdriver\chromedriver.exe', hide=False):    
    try:        
        service=Service(chromeDriver)
        options=webdriver.ChromeOptions()  
        userAgent = UserAgent().random   # 為了搞定反爬蟲
        # print(userAgent )
        options.add_argument(f'user-agent= {userAgent}')
        if hide:
            options.add_argument('--headless')
        chrome= webdriver.Chrome(service= service, options=options)
        chrome.implicitly_wait(15)
        chrome.get(url)
        return chrome
    except Exception as e:
        print(e, "是出了什麼問題...?")    
    return "Nothing"
def find_element(chrome, xpath):
    try:
        return chrome.find_element(By.XPATH, xpath)
    except Exception as e:
        print(e, "xpath怪怪的@@")
    return "Nothing"

In [17]:
url = "https://www.google.com/maps/"
datas = pd.read_csv("Taipei_to_hots_copy1.csv", encoding = "utf-8-sig")
print(datas)

     Unnamed: 0.1  Unnamed: 0   ID            shopName  \
0               0           0    0               紫明溪步道   
1               1           1    1           冷水坑公共溫泉浴室   
2               2           2    2  中國信託文薈館CTBC Museum   
3               3           3    3                大湖公園   
4               4           4    4                和興炭坑   
..            ...         ...  ...                 ...   
150           150         150  150     璐露野生活Luluyelife   
151           151         151  151          花博公園（新生園區）   
152           152         152  152             劍潭山親山步道   
153           153         153  153            臺北市兒童新樂園   
154           154         154  154      華山1914文化創意產業園區   

                        shopAddress          shopPhone  \
0            台北市北投區中興路12號附近（陽明書屋附近）       02-2759-3001   
1                 台北市士林區菁山路101巷175號       02-2861-0036   
2    台北市南港區經貿二路168號1樓（中國信託金融園區A棟1樓）  02-3327-7777#1099   
3                    台北市內湖區成功路五段31號  02-3327-7777#1099   
4            

In [35]:
chrome = get_chrome(url, hide = False)
chrome.refresh()
#網頁最大化
chrome.maximize_window()
xpath1 = "/html/body/div[1]/div[3]/div[8]/div[3]/div[1]/div[1]/div/div[2]/form/input"
element = find_element(chrome, xpath1)
element.clear()
element.click()
element.send_keys(datas["shopName"][0] + "\n")

In [34]:
chrome.quit()

#### 有選項

In [36]:
if "Nv2PK Q2HXcd THOPZb" in str(chrome.page_source):
    soup = BeautifulSoup(chrome.page_source, "html.parser")
    num = -1
    for number in soup.find_all("div", class_ = "Nv2PK Q2HXcd THOPZb"):
        number_text = eval(number.find("span", class_ = "UY7F9").text)
        if number_text > num:
            num = number_text
            url_new = number.find("a", class_ = "hfpxzc").get("href")
    chrome.quit()
chrome = get_chrome(url_new, hide = False)
time.sleep(0.5)
#網頁最大化
chrome.maximize_window()
chrome.refresh()
time.sleep(0.5)
element = chrome.find_element(By.CLASS_NAME, 'wNNZR')
element.click()
time.sleep(0.5)
comment_roller = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]"
pane = chrome.find_element(By.XPATH, comment_roller)
chrome.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", pane)
time.sleep(0.5)
# 不一定要有這個
comment_roller = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]"
pane1 = chrome.find_element(By.XPATH, comment_roller)
chrome.execute_script("arguments[0].scrollTop = 0", pane1)
time.sleep(0.5)
soup2 = BeautifulSoup(chrome.page_source, "html.parser")
comments = soup2.find_all("div", class_ = "jftiEf fontBodyMedium")
print(len(comments))
# print(comments)
time.sleep(2)
# numbers_w8nwRe = [one.find(class_ = "w8nwRe") for one in comments if "w8nwRe" in str(one)]
for i in comments:
    if "w8nwRe" in str(i): 
        element = chrome.find_element(By.CLASS_NAME, 'w8nwRe')
        element.click()
time.sleep(0.5)
comment_roller = "/html/body/div[2]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]"
pane1 = chrome.find_element(By.XPATH, comment_roller)
chrome.execute_script("arguments[0].scrollTop = 0", pane1)
time.sleep(0.5)
soup2 = BeautifulSoup(chrome.page_source, "html.parser")
comments = soup2.find_all("div", class_ = "jftiEf fontBodyMedium")
print(len(comments))
print(comments)

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=121.0.6167.189)
Stacktrace:
	GetHandleVerifier [0x00007FF718AF5E42+3538674]
	(No symbol) [0x00007FF718714C02]
	(No symbol) [0x00007FF7185C599D]
	(No symbol) [0x00007FF71860D406]
	(No symbol) [0x00007FF718601950]
	(No symbol) [0x00007FF71862F0AA]
	(No symbol) [0x00007FF71860124A]
	(No symbol) [0x00007FF71862F2C0]
	(No symbol) [0x00007FF71864BDE3]
	(No symbol) [0x00007FF71862EE53]
	(No symbol) [0x00007FF7185FF514]
	(No symbol) [0x00007FF718600631]
	GetHandleVerifier [0x00007FF718B26CAD+3738973]
	GetHandleVerifier [0x00007FF718B7C506+4089270]
	GetHandleVerifier [0x00007FF718B74823+4057299]
	GetHandleVerifier [0x00007FF718845C49+720121]
	(No symbol) [0x00007FF71872126F]
	(No symbol) [0x00007FF71871C304]
	(No symbol) [0x00007FF71871C432]
	(No symbol) [0x00007FF71870BD04]
	BaseThreadInitThunk [0x00007FFA319E257D+29]
	RtlUserThreadStart [0x00007FFA3314AA58+40]


In [ ]:
soup2 = BeautifulSoup(chrome.page_source, "html.parser")
"全文" in str(soup2)

In [ ]:
soup2 = BeautifulSoup(chrome.page_source, "html.parser")
comments = soup2.find_all("div", class_ = "jftiEf fontBodyMedium")
print(len(comments))
print(comments)

In [ ]:
data = []
for com in comments:
    temp = []
    if "wiI7pd" in str(com):            
        people = "姓名: " + com.find(class_ = "d4r55").text
        comment = "評論 : " + com.find(class_ = "wiI7pd").text
        temp.append([people, comment])
    data.append(temp)

In [ ]:
data

In [ ]:
len(data)

In [ ]:
chrome.quit()

###### <button class="w8nwRe kyuRq" aria-expanded="false" aria-controls="ChdDSUhNMG9nS0VJQ0FnSUNOeHRPWW1nRRAB" 
data-review-id="ChdDSUhNMG9nS0VJQ0FnSUNOeHRPWW1nRRAB" 
jslog="63707; track:click;metadata:WyIwYWhVS0V3anZ5dFdNNk1XRUF4VTRTRjRFSFVzYUNHUVEwcE1GQ0d3b0FRIl0=" 
aria-label="顯示更多" jsaction="pane.wfvdle91.review.expandReview">全文</button>


###### <button class="w8nwRe kyuRq" aria-expanded="false" aria-controls="ChdDSUhNMG9nS0VJQ0FnSURHNHZ1QS1nRRAB" 
data-review-id="ChdDSUhNMG9nS0VJQ0FnSURHNHZ1QS1nRRAB" 
jslog="63707; track:click;metadata:WyIwYWhVS0V3akpwOGY2cWNhRUF4V2tWbDRFSFMzQ0M1MFEwcE1GQ0NZb0FRIl0=" 
aria-label="顯示更多" jsaction="pane.wfvdle102.review.expandReview">全文</button>